*   **Año:** [INSERTAR AÑO]
*   **Alumno/a:** [INSERTAR NOMBRE]
*   **Legajo:** [LEGAJO]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [10]:
import pydantic
from pydantic import BaseModel, validator
from datetime import date
from typing import Optional

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [11]:
class Client(BaseModel):
    dni: int
    nationality: str
    registration_date: Optional[date] = None

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [12]:
client = Client(dni=39755010, nationality='Argentina')
print(client)

dni=39755010 nationality='Argentina' registration_date=None


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [13]:
try:
    client_invalid_dni = Client(dni='39755010', nationality='Argentina')
except ValueError as e:
    print(f"Error: {e}")

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [14]:
class Client(BaseModel):
    dni: int
    nationality: str
    registration_date: Optional[date] = None
    
    @validator('dni')
    def dni_must_be_positive(cls, v):
        if v < 0:
            raise ValueError('DNI must be positive')
        return v

    @validator('nationality')
    def nationality_must_be_valid(cls, v):
        valid_nationalities = ['Argentina', 'Brasil', 'Chile', 'Uruguay']  # Puedes agregar más
        if v not in valid_nationalities:
            raise ValueError(f"Nationality '{v}' is not valid")
        return v

    @validator('registration_date', pre=True, always=True)
    def check_registration_date(cls, v):
        if v and v > date.today():
            raise ValueError("Registration date can't be in the future")
        return v

/var/folders/3y/mm6bmkvd4nd8_j46k_7rqdgr0000gn/T/ipykernel_15961/880137703.py:6: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('dni')
/var/folders/3y/mm6bmkvd4nd8_j46k_7rqdgr0000gn/T/ipykernel_15961/880137703.py:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('nationality')
/var/folders/3y/mm6bmkvd4nd8_j46k_7rqdgr0000gn/T/ipykernel_15961/880137703.py:19: PydanticDeprecatedSince20: Pydantic V1 style `@valida

In [15]:
client_valid = Client(dni=39755010, nationality='Argentina', registration_date=date.today())
print(client_valid)

dni=39755010 nationality='Argentina' registration_date=datetime.date(2024, 10, 8)


In [16]:
try:
    client_invalid_nationality = Client(dni=39755010, nationality='Peru')
except ValueError as e:
    print(f"Error: {e}")

Error: 1 validation error for Client
nationality
  Value error, Nationality 'Peru' is not valid [type=value_error, input_value='Peru', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [17]:
try:
    client_invalid_dni = Client(dni=-12345, nationality='Argentina')
except ValueError as e:
    print(f"Error: {e}")

Error: 1 validation error for Client
dni
  Value error, DNI must be positive [type=value_error, input_value=-12345, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [18]:
try:
    client_invalid_date = Client(dni=39755010, nationality='Argentina', registration_date=date(2050, 1, 1))
except ValueError as e:
    print(f"Error: {e}")

Error: 1 validation error for Client
registration_date
  Value error, Registration date can't be in the future [type=value_error, input_value=datetime.date(2050, 1, 1), input_type=date]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.